In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [2]:
!cd dataset_creation

In [4]:
from pathlib import Path
from datetime import date 

folder =  Path("dataset_creation") / "base"

In [5]:
from preprocess_utils import load_jsonb, save_jsonb 
from urllib.parse import urlparse
for art in load_jsonb("final.jsonb"):
    url = art["url"]
    parsed = urlparse(url).netloc.split(".")
    server = parsed[-1]
    if server == "cz":
        server = parsed[-2]

    save_jsonb([art], folder / f"{server}.jsonb", "a", show_progress=False)



    



100%|██████████| 3193720/3193720 [1:33:08<00:00, 571.51it/s]


In [13]:
# Add from cz:403 to aktualne becauose of the same domain

# for art in load_jsonb(folder / "cz:443.jsonb"):
#     save_jsonb([art], folder / "aktualne.jsonb", "a", show_progress=False)

# Remove cz:443
import os
os.remove(folder / "cz:443.jsonb")
os.remove(folder / "ihned.jsonb")


In [7]:
from preprocess_utils import articles_num
print(list(articles_num(folder)))

[('ihned.jsonb', 31643), ('aktualne.jsonb', 165799), ('cz:443.jsonb', 1), ('denik.jsonb', 1636639), ('irozhlas.jsonb', 201681), ('novinky.jsonb', 549661), ('idnes.jsonb', 530855), ('seznamzpravy.jsonb', 77442)]


In [3]:

from typing import Callable, List
from filtering import create_filter_by_stats, create_filter_by_cz_lang, create_config, create_tokenized_filter
from preprocess_utils import load_jsonb, save_jsonb
from postprocessing_utils import postprocess_authors, filter_author, add_server, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date, as_Article, LowerTopXWhiten ,JSONArticleEncoder
filtered_folder = Path("article_json_filtered")

In [2]:
# Custom config for each server
from filtering import between
configs = {
    "idnes": {},
    "denik": {},
    "aktualne": {},
    "irozhlas": {},
    "seznamzpravy": {},
    "novinky": {},
}

default = {
    "article_length": between(300, None),
    "avg_word_length": between(4.0, None),
    "num_words_ratio": between(0.11, 0.22),
    "headline_length": between(20, None),
    "brief_length": between(40, None),
    "non_alpha_ratio": between(None, 0.045),
    "date": between(date(2000, 1, 1), date(2022, 8, 31)),
}

In [8]:

from functools import reduce
servers = [f for f in folder.iterdir()]

ratio = 0.75
for f in servers:
    filters = [
        #TODO run without tokenized filter
        create_filter_by_cz_lang(),
        create_filter_by_stats(create_config(configs[f.stem], default)),
        create_tokenized_filter(lambda x: len(x) <= 4, "category"),
        # Max 4 words in category
    ]

    augmentations: List[Callable[[dict], dict]] = [add_server(f.stem), filter_author, postprocess_authors, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date]


    articles = load_jsonb(f)
    articles = filter(lambda x: all(f(x) for f in filters), articles)


    #TODO Apply as df filters as could be faster    
    articles = (reduce(lambda x, f: f(x), augmentations, article) for article in articles)
    


    as_articles = (as_Article(article) for article in articles)
    save_jsonb(as_articles, filtered_folder / f"{f.stem}.jsonb", encoder=JSONArticleEncoder, show_progress=False)

         



    

100%|██████████| 77442/77442 [06:41<00:00, 192.84it/s]


In [9]:

from preprocess_utils import articles_num
print(list(articles_num(filtered_folder)))

[('seznamzpravy.jsonb', 67378), ('idnes.jsonb', 314479), ('irozhlas.jsonb', 172151), ('aktualne.jsonb', 118210), ('denik.jsonb', 1185135), ('novinky.jsonb', 371566)]


In [10]:
# Create splits 70/20/10 by date
from datetime import date
import random
output_folder = Path("final_dataset")
servers = [f for f in filtered_folder.iterdir()]
#servers = [fixed_folder / "seznamzpravy.jsonb"]

all_articles = (article for f in servers for article in load_jsonb(f))
none_articles = (article for article in all_articles if article["date"] is None)
articles = (article for article in all_articles if article["date"] is not None)
sorted_articles = sorted(articles, key=lambda x: date.fromisoformat(x["date"]))
# Randomly insert none articles
for none_art in none_articles:
    index = random.randint(0, len(sorted_articles))
    sorted_articles.insert(index, none_art)

train = sorted_articles[:int(len(sorted_articles) * 0.8)]
dev = sorted_articles[int(len(sorted_articles) * 0.8):int(len(sorted_articles) * 0.9)]
test = sorted_articles[int(len(sorted_articles) * 0.9):]

save_jsonb(train, output_folder / "train.jsonb", show_progress=True)
save_jsonb(dev, output_folder / "dev.jsonb", show_progress=True)
save_jsonb(test, output_folder / "test.jsonb", show_progress=True)



100%|██████████| 222746/222746 [00:26<00:00, 8385.78it/s]


In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
